# Replicating iwatobipen solubility results with DeepChem #
    - Matt Robinson

https://iwatobipen.wordpress.com/2019/02/01/try-gcn-qspr-with-pytorch-based-graph-library-rdkit-pytorch-dgl/

The prolific and well-known chemoinformatics blogger *iwatobipen* released his result using a graph convolutional network similarly built with pytorch and dgl. Here we compare his results to those obtained with our own gcn built with pytorch and gcn. Our gcn is built includes a few more features somewhat emulating the structure of the DeepChem gcn.

The dataset is a solubility dataset available directly from RDKIT. 

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import deepchem as dc
from deepchem.models.tensorgraph.models.graph_models import GraphConvModel

/Users/matthewrobinson/anaconda/envs/deepchem-env/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
from rdkit import Chem
from rdkit.Chem import RDConfig
import os

In [6]:
solcls = {'(A) low':0, '(B) medium':1, '(C) high':2} # note the 3 classes

train_mols = [m for m in Chem.SDMolSupplier(os.path.join(RDConfig.RDDocsDir,'Book/data/solubility.train.sdf'))]
train_smiles = [Chem.MolToSmiles(x) for x in train_mols]
train_y = [solcls[m.GetProp('SOL_classification')] for m in train_mols]

test_mols = [m for m in Chem.SDMolSupplier(os.path.join(RDConfig.RDDocsDir,'Book/data/solubility.test.sdf'))]
test_smiles = [Chem.MolToSmiles(x) for x in test_mols]
test_y = [solcls[m.GetProp('SOL_classification')] for m in test_mols]

In [7]:
trainset_df = pd.DataFrame({'smiles': train_smiles, 'labels': train_y})
trainset_df.to_csv('solubility_classification_train.csv',index=False)

In [8]:
testset_df = pd.DataFrame({'smiles': test_smiles, 'labels': test_y})
testset_df.to_csv('solubility_classification_test.csv',index=False)

In [9]:
print('training set size: ', len(train_smiles))
print('testing_set_size: ', len(test_smiles))

('training set size: ', 1025)
('testing_set_size: ', 257)


In [10]:
graph_featurizer = dc.feat.graph_features.ConvMolFeaturizer()
loader = dc.data.data_loader.CSVLoader(tasks=['labels'],
                                       smiles_field="smiles",
                                       featurizer=graph_featurizer)

trainset = loader.featurize('./solubility_classification_train.csv')
testset = loader.featurize('./solubility_classification_test.csv')

Loading raw samples now.
shard_size: 8192
About to start loading CSV from ./solubility_classification_train.csv
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
TIMING: featurizing shard 0 took 1.672 s
TIMING: dataset construction took 3.960 s
Loading dataset from disk.
Loading raw samples now.
shard_size: 8192
About to start loading CSV from ./solubility_classification_test.csv
Loading shard 1 of size 8192.
Featurizing sample 0
TIMING: featurizing shard 0 took 0.396 s
TIMING: dataset construction took 0.973 s
Loading dataset from disk.


In [11]:
model = GraphConvModel(n_tasks=1, n_classes=len(solcls), mode='classification',
                       tensorboard=True,  model_dir='solubility_models/',
                       dropout=0.2, graph_conv_layers=[64,64])

In [12]:
%%time
# fit for 100 epochs
for _ in range(20):
    train_loss = model.fit(trainset, nb_epoch=5)
    print('train_loss: ', train_loss)

/Users/matthewrobinson/anaconda/envs/deepchem-env/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


('train_loss: ', 85.43233614834872)
('train_loss: ', 66.33962163058195)
('train_loss: ', 57.29524721665816)
('train_loss: ', 50.515527257052334)
('train_loss: ', 45.40156693892045)
('train_loss: ', 42.42582628076727)
('train_loss: ', 38.53086421272972)
('train_loss: ', 35.42941387349909)
('train_loss: ', 33.07965602007779)
('train_loss: ', 30.58448325070468)
('train_loss: ', 28.680832316658712)
('train_loss: ', 26.97158300226385)
('train_loss: ', 25.668372518366034)
('train_loss: ', 23.405790814486416)
('train_loss: ', 23.75758847323331)
('train_loss: ', 21.848122774470937)
('train_loss: ', 21.01615975553339)
('train_loss: ', 20.152980336275967)
('train_loss: ', 17.974479846332382)
('train_loss: ', 17.96369844783436)
CPU times: user 12min 19s, sys: 1min 55s, total: 14min 14s
Wall time: 11min 12s


In [13]:
train_scores = model.evaluate(
                trainset,
                [dc.metrics.Metric(dc.metrics.accuracy_score)]
                )
print('train scores')
print(train_scores)

computed_metrics: [0.9014634146341464]
train scores
{'accuracy_score': 0.9014634146341464}


In [14]:
test_scores = model.evaluate(
                testset,
                [dc.metrics.Metric(dc.metrics.accuracy_score)]
                )
print('test scores')
print(test_scores)

computed_metrics: [0.7704280155642024]
test scores
{'accuracy_score': 0.7704280155642024}


In [21]:
def evaluate_dc_classifier(model, test_ds, classes=[0,1]):
    "evaluate a DeepChem model for classification"
    
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import roc_auc_score, auc, roc_curve
    from sklearn.preprocessing import label_binarize
    
    def bs_roc_auc_score(y_true, y_prob, n_boostraps=1000):
        "code to bootstrap the auc score: copied from Ogrisel's SO answer"

        n_bootstraps = 1000
        rng_seed = 42  # control reproducibility
        bootstrapped_scores = []

        rng = np.random.RandomState(rng_seed)
        for i in range(n_bootstraps):
            # bootstrap by sampling with replacement on the prediction indices
            indices = rng.randint(0, len(y_prob) - 1, len(y_prob))
            indices = [int(idx) for idx in indices]
            y_true = np.array(y_true)
            y_prob = np.array(y_prob)
            if len(np.unique(y_true[indices])) < 2:
                # We need at least one positive and one negative sample for ROC AUC
                # to be defined: reject the sample
                continue

            score = roc_auc_score(y_true[indices], y_prob[indices])
            bootstrapped_scores.append(score)

        sorted_scores = np.array(bootstrapped_scores)
        sorted_scores.sort()

        # Computing the lower and upper bound of the 90% confidence interval
        # You can change the bounds percentiles to 0.025 and 0.975 to get
        # a 95% confidence interval instead.
        confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
        confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
        return [confidence_lower, confidence_upper]

    y_true = list(testset.y[:,0].astype(int))
    
    prob_arr = np.array([x[0] for x in model.predict(testset)])
    y_pred = list(np.argmax(prob_arr, axis=1))

    print('accuracy: ', accuracy_score(y_true, y_pred))
    print('classification report: ')
    print(classification_report(y_true, y_pred))

    if len(classes) == 2:
        y_prob = list(prob_arr[:,-1])
        print('roc-auc: ', roc_auc_score(y_true, y_prob))
        print('bootstrapped roc-auc: ', bs_roc_auc_score(y_true, y_prob))

    else:
        y_test = label_binarize(y_true, classes=classes)
        n_classes = y_test.shape[1]

        # Compute ROC curve and ROC area for each class
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        bs_roc_auc = dict()
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_test[:, i], prob_arr[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
            bs_roc_auc[i] = bs_roc_auc_score(y_test[:, i], prob_arr[:, i])
        
        fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), prob_arr.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        bs_roc_auc['micro'] = bs_roc_auc_score(y_test.ravel(), prob_arr.ravel())

        print("micro auc score and score for each class: ")
        for key in roc_auc:
            print(key,' : ', roc_auc[key])
        print("bootstrapped micro auc score and score for each class: ")
        for key in bs_roc_auc:
            print(key,' : ', bs_roc_auc[key])

In [22]:
evaluate_dc_classifier(model, testset, classes=[0,1,2])

('accuracy: ', 0.7704280155642024)
classification report: 
             precision    recall  f1-score   support

          0       0.91      0.71      0.80       102
          1       0.73      0.77      0.75       115
          2       0.66      0.95      0.78        40

avg / total       0.79      0.77      0.77       257

micro auc score and score for each class: 
(0, ' : ', 0.9384566729917774)
(1, ' : ', 0.8349663196570728)
(2, ' : ', 0.9705069124423963)
('micro', ' : ', 0.9161153083316931)
bootstrapped micro auc score and score for each class: 
(0, ' : ', [0.9131093544137022, 0.9581406972385355])
(1, ' : ', [0.7892857142857144, 0.8744976251370113])
(2, ' : ', [0.9526205450733752, 0.9853820598006644])
('micro', ' : ', [0.8999318710288956, 0.9313243761996162])
